In [3]:
from sklearn.linear_model import RANSACRegressor

In [4]:
ransac = RANSACRegressor(
    LinearRegression(),
    max_trials=100, #default is 100
    min_samples=0.95, 
    residual_threshold=None, #default is None
    random_state=123
)

In [ ]:
ransac.fit()